In [1]:
import catboost as cb
import gc
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import sys
sys.path.append("../")
import warnings
warnings.simplefilter("ignore")
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, fbeta_score, make_scorer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
from utils.common import (
    sigmoid, pad_column_name
)
from utils.constants import *
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric
)
from utils.eda_helpers import (
    plot_missing_proportion_barchart, 
    get_cols
)
from utils.extraction_helpers import read_file
from utils.feature_group import (
    CATEGORY_COLUMNS, CONTINUOUS_COLUMNS, NON_FEATURE_COLUMNS
)
from utils.feature_engineering_helpers import feature_gen_pipeline

In [3]:
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
from cycler import cycler
from IPython.display import display
from colorama import Fore, Back, Style
plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
plt.rcParams['text.color'] = 'w'

In [4]:
%load_ext autoreload
%autoreload

### Read Data

In [5]:
train_agg = read_file(f"{PROCESSED_DATA_PATH}/v2/train_agg.pkl")


Shape of data: (458913, 4083)


In [6]:
labels = read_file(f"{RAW_DATA_PATH}/train_labels.csv")

Shape of data: (458913, 2)


In [7]:
# %%time
# train = pd.concat([
#     spend_payment_df, 
#     risk_df.loc[:, get_cols(risk_df, "R_")]
# ], axis=1)

In [8]:
# train.loc[:, get_cols(train, "R_")] = train.loc[:, get_cols(train, "R_")].fillna(-127)

#### Generate Feature

In [9]:
# %%time
# train_agg, keep_column = feature_gen_pipeline(train)

In [10]:
# train_agg.head(3)

In [11]:
# len(features_to_drop)
# train_agg = train_agg.drop(columns=list(features_to_drop), errors="ignore")

In [12]:
target = labels["target"].values

In [13]:
%%time
train_agg = train_agg.drop(columns=NON_FEATURE_COLUMNS + ["target"], errors="ignore")
gc.collect()

CPU times: user 1.33 s, sys: 9.64 s, total: 11 s
Wall time: 25.2 s


4

In [17]:
# feature_imp = dict(zip(model.feature_names_, model.feature_importances_))

In [ ]:
noob_features = [k for k, v in feature_imp.items() if v < 0.005]

In [ ]:
len(noob_features)

In [ ]:
train_agg = train_agg.drop(noob_features, errors="ignore")

### Tune Catboost

In [21]:
def objective(trial):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1020)

    param = {
        "objective": "Logloss",
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.03, 0.1, log=True),
        "depth": trial.suggest_int("depth", 8, 15),
        "random_strength": trial.suggest_float("random_strength", 1.1, 1.5),
        "class_weights": trial.suggest_categorical("class_weights", [[1, 1.25], [1, 1.5], [1, 2]]),
        "n_estimators": trial.suggest_categorical("n_estimators", [500, 750, 1000]),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 2, 8, log=True),
        "min_data_in_leaf": trial.suggest_categorical("min_data_in_leaf", [1024, 2048, 3072]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.03, 0.05, 0.08]),
        "max_bin": trial.suggest_categorical("max_bin", [63, 127, 255]),
        "used_ram_limit": "16gb",
        "bootstrap_type": "MVS",
        # "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "grow_policy": "Depthwise"
    }

    if param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.6, 0.85)
    
    print(param)
    train_score_list, val_score_list = [], []
    for fold, (idx_tr, idx_va) in zip(range(1, 5+1), kf.split(train_agg, target)):
        fold = str(fold)
        X_train, y_train = train_agg.iloc[idx_tr], target[idx_tr]
        X_val, y_val = train_agg.iloc[idx_va], target[idx_va]
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)
            model = cb.CatBoostClassifier(**param)
            model.fit(
                X_train, 
                y_train, 
                eval_set=[(X_val, y_val)], 
                verbose=0, 
                cat_features=cat_features,
                early_stopping_rounds=100
            )
        y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
        train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
        train_data, X_train, y_train = None, None, None
        y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
        val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
        valid_data, X_val, y_val = None, None, None
        train_score_list.append(train_score)
        val_score_list.append(val_score)
        # if val_score > best_scores_json["validation"][fold]:
        #     best_scores_json["train"][fold] = train_score
        #     best_scores_json["validation"][fold] = val_score
        #     with open(f'{CURRENT_EXP_PATH}/best_scores.json', "w") as outfile:
        #         json.dump(best_scores_json, outfile)
        #     joblib.dump(model, f'{CURRENT_EXP_PATH}/models/model{fold}.pkl')
        # elif np.mean(train_score_list) >= np.mean(list(best_scores_json["train"].values())) + 0.02:
        #     print(f"Train score too high (overfitting), start a new trial")
        #     return np.mean(val_score_list)
        joblib.dump(model, f'{EXP_PATH}/catboost_exp/models/model{fold}_seed1020_{val_score:.5f}.pkl')
        print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
        print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
        print(f"Clear cache {gc.collect()}")
        
    return np.mean(val_score_list)

In [22]:
# study = joblib.load(f"{CURRENT_EXP_PATH}/optuna_study.pkl")

In [23]:
study = optuna.create_study(direction="maximize")

[I 2022-08-22 03:05:12,622] A new study created in memory with name: no-name-b960f36f-1831-4b57-aeab-3c18afed776a


In [24]:
study.optimize(objective, n_trials=20)

{'objective': 'Logloss', 'colsample_bylevel': 0.07996767207704472, 'depth': 15, 'random_strength': 1.1383999051610745, 'class_weights': [1, 1.5], 'n_estimators': 1000, 'l2_leaf_reg': 3.385613504354621, 'min_data_in_leaf': 1024, 'learning_rate': 0.05, 'max_bin': 255, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise'}
Fold 1 | Train Score = 0.93139 (0.9748, 0.8880)
Fold 1 | Val Score = 0.79156 (0.9216, 0.6615)
Clear cache 0
Fold 2 | Train Score = 0.93222 (0.9750, 0.8895)
Fold 2 | Val Score = 0.79208 (0.9229, 0.6613)
Clear cache 0
Fold 3 | Train Score = 0.93716 (0.9767, 0.8977)
Fold 3 | Val Score = 0.79357 (0.9243, 0.6629)
Clear cache 0
Fold 4 | Train Score = 0.93660 (0.9766, 0.8966)
Fold 4 | Val Score = 0.79715 (0.9241, 0.6702)
Clear cache 0


[I 2022-08-22 04:22:40,531] Trial 0 finished with value: 0.7939853451203132 and parameters: {'colsample_bylevel': 0.07996767207704472, 'depth': 15, 'random_strength': 1.1383999051610745, 'class_weights': [1, 1.5], 'n_estimators': 1000, 'l2_leaf_reg': 3.385613504354621, 'min_data_in_leaf': 1024, 'learning_rate': 0.05, 'max_bin': 255}. Best is trial 0 with value: 0.7939853451203132.


Fold 5 | Train Score = 0.93520 (0.9759, 0.8945)
Fold 5 | Val Score = 0.79557 (0.9251, 0.6661)
Clear cache 0
{'objective': 'Logloss', 'colsample_bylevel': 0.06861115112895604, 'depth': 10, 'random_strength': 1.1496130055769387, 'class_weights': [1, 1.25], 'n_estimators': 750, 'l2_leaf_reg': 4.659639396354976, 'min_data_in_leaf': 2048, 'learning_rate': 0.05, 'max_bin': 127, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise'}
Fold 1 | Train Score = 0.88410 (0.9575, 0.8107)
Fold 1 | Val Score = 0.79177 (0.9220, 0.6615)
Clear cache 0
Fold 2 | Train Score = 0.88272 (0.9571, 0.8084)
Fold 2 | Val Score = 0.79258 (0.9232, 0.6620)
Clear cache 0
Fold 3 | Train Score = 0.88032 (0.9561, 0.8046)
Fold 3 | Val Score = 0.79460 (0.9246, 0.6646)
Clear cache 0
Fold 4 | Train Score = 0.87961 (0.9561, 0.8032)
Fold 4 | Val Score = 0.79744 (0.9242, 0.6707)
Clear cache 0


[I 2022-08-22 05:15:06,196] Trial 1 finished with value: 0.7944883370440556 and parameters: {'colsample_bylevel': 0.06861115112895604, 'depth': 10, 'random_strength': 1.1496130055769387, 'class_weights': [1, 1.25], 'n_estimators': 750, 'l2_leaf_reg': 4.659639396354976, 'min_data_in_leaf': 2048, 'learning_rate': 0.05, 'max_bin': 127}. Best is trial 1 with value: 0.7944883370440556.


Fold 5 | Train Score = 0.88036 (0.9561, 0.8046)
Fold 5 | Val Score = 0.79605 (0.9252, 0.6669)
Clear cache 0
{'objective': 'Logloss', 'colsample_bylevel': 0.04765602557412835, 'depth': 14, 'random_strength': 1.4548330556514888, 'class_weights': [1, 1.5], 'n_estimators': 1000, 'l2_leaf_reg': 2.217173179895009, 'min_data_in_leaf': 2048, 'learning_rate': 0.05, 'max_bin': 255, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise'}
Fold 1 | Train Score = 0.91501 (0.9694, 0.8607)
Fold 1 | Val Score = 0.79160 (0.9218, 0.6614)
Clear cache 0
Fold 2 | Train Score = 0.90037 (0.9645, 0.8362)
Fold 2 | Val Score = 0.79239 (0.9232, 0.6616)
Clear cache 0
Fold 3 | Train Score = 0.92197 (0.9716, 0.8723)
Fold 3 | Val Score = 0.79625 (0.9246, 0.6679)
Clear cache 0
Fold 4 | Train Score = 0.90053 (0.9645, 0.8366)
Fold 4 | Val Score = 0.79586 (0.9244, 0.6673)
Clear cache 0


...

[I 2022-08-22 09:56:50,239] Trial 7 finished with value: 0.7914996180678676 and parameters: {'colsample_bylevel': 0.0303377100038264, 'depth': 15, 'random_strength': 1.1431218357244004, 'class_weights': [1, 2], 'n_estimators': 1000, 'l2_leaf_reg': 5.024136281118513, 'min_data_in_leaf': 2048, 'learning_rate': 0.08, 'max_bin': 255}. Best is trial 6 with value: 0.7951733682088421.


{'objective': 'Logloss', 'colsample_bylevel': 0.04911094180345822, 'depth': 11, 'random_strength': 1.113375484463458, 'class_weights': [1, 1.5], 'n_estimators': 1000, 'l2_leaf_reg': 5.884720669253448, 'min_data_in_leaf': 2048, 'learning_rate': 0.08, 'max_bin': 127, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise'}
Fold 1 | Train Score = 0.87623 (0.9559, 0.7966)
Fold 1 | Val Score = 0.78906 (0.9207, 0.6574)
Clear cache 0
Fold 2 | Train Score = 0.88296 (0.9582, 0.8077)
Fold 2 | Val Score = 0.78920 (0.9224, 0.6560)
Clear cache 0
Fold 3 | Train Score = 0.91180 (0.9678, 0.8558)
Fold 3 | Val Score = 0.79381 (0.9237, 0.6639)
Clear cache 0
Fold 4 | Train Score = 0.88854 (0.9603, 0.8168)
Fold 4 | Val Score = 0.79479 (0.9241, 0.6655)
Clear cache 0
Fold 5 | Train Score = 0.87173 (0.9544, 0.7891)
Fold 5 | Val Score = 0.79453 (0.9249, 0.6642)
Clear cache 0


[I 2022-08-22 10:33:05,706] Trial 8 finished with value: 0.792276153840165 and parameters: {'colsample_bylevel': 0.04911094180345822, 'depth': 11, 'random_strength': 1.113375484463458, 'class_weights': [1, 1.5], 'n_estimators': 1000, 'l2_leaf_reg': 5.884720669253448, 'min_data_in_leaf': 2048, 'learning_rate': 0.08, 'max_bin': 127}. Best is trial 6 with value: 0.7951733682088421.


{'objective': 'Logloss', 'colsample_bylevel': 0.07380057074910772, 'depth': 12, 'random_strength': 1.3006359729951702, 'class_weights': [1, 2], 'n_estimators': 750, 'l2_leaf_reg': 2.313422208449659, 'min_data_in_leaf': 1024, 'learning_rate': 0.05, 'max_bin': 127, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise'}


KeyboardInterrupt: 

In [25]:
joblib.dump(study, f"{EXP_PATH}/catboost_exp/catboost_optuna_study.pkl")

['../experiments/catboost_exp/catboost_optuna_study.pkl']

In [169]:
study.optimize(objective, n_trials=20)

{'objective': 'Logloss', 'colsample_bylevel': 0.04102624715853323, 'depth': 11, 'random_strength': 1.2633436553615531, 'class_weights': [1, 2], 'n_estimators': 1000, 'l2_leaf_reg': 7.532364531307307, 'min_data_in_leaf': 3072, 'learning_rate': 0.03, 'max_bin': 255, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise'}
Fold 1 | Train Score = 0.85778 (0.9509, 0.7646)
Fold 1 | Val Score = 0.79111 (0.9217, 0.6606)
Clear cache 85219
Fold 2 | Train Score = 0.85752 (0.9509, 0.7641)
Fold 2 | Val Score = 0.79243 (0.9229, 0.6620)
Clear cache 0
Fold 3 | Train Score = 0.85710 (0.9506, 0.7636)
Fold 3 | Val Score = 0.79440 (0.9244, 0.6644)
Clear cache 0
Fold 4 | Train Score = 0.85682 (0.9505, 0.7632)
Fold 4 | Val Score = 0.79502 (0.9245, 0.6656)
Clear cache 0


[I 2022-08-20 08:36:29,594] Trial 0 finished with value: 0.7938155963625977 and parameters: {'colsample_bylevel': 0.04102624715853323, 'depth': 11, 'random_strength': 1.2633436553615531, 'class_weights': [1, 2], 'n_estimators': 1000, 'l2_leaf_reg': 7.532364531307307, 'min_data_in_leaf': 3072, 'learning_rate': 0.03, 'max_bin': 255}. Best is trial 0 with value: 0.7938155963625977.


Fold 5 | Train Score = 0.85823 (0.9511, 0.7653)
Fold 5 | Val Score = 0.79612 (0.9253, 0.6669)
Clear cache 0
{'objective': 'Logloss', 'colsample_bylevel': 0.05402675785295788, 'depth': 11, 'random_strength': 1.219098359987579, 'class_weights': [1, 1.25], 'n_estimators': 750, 'l2_leaf_reg': 4.280904816220738, 'min_data_in_leaf': 3072, 'learning_rate': 0.03, 'max_bin': 63, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise'}
Fold 1 | Train Score = 0.85091 (0.9461, 0.7557)
Fold 1 | Val Score = 0.79088 (0.9211, 0.6606)
Clear cache 0
Fold 2 | Train Score = 0.85169 (0.9459, 0.7575)
Fold 2 | Val Score = 0.79087 (0.9226, 0.6592)
Clear cache 0
Fold 3 | Train Score = 0.85210 (0.9462, 0.7580)
Fold 3 | Val Score = 0.79501 (0.9240, 0.6660)
Clear cache 0
Fold 4 | Train Score = 0.85088 (0.9458, 0.7560)
Fold 4 | Val Score = 0.79431 (0.9237, 0.6649)
Clear cache 0


[I 2022-08-20 09:22:46,751] Trial 1 finished with value: 0.793635049670822 and parameters: {'colsample_bylevel': 0.05402675785295788, 'depth': 11, 'random_strength': 1.219098359987579, 'class_weights': [1, 1.25], 'n_estimators': 750, 'l2_leaf_reg': 4.280904816220738, 'min_data_in_leaf': 3072, 'learning_rate': 0.03, 'max_bin': 63}. Best is trial 0 with value: 0.7938155963625977.


Fold 5 | Train Score = 0.85071 (0.9457, 0.7557)
Fold 5 | Val Score = 0.79711 (0.9250, 0.6692)
Clear cache 0
{'objective': 'Logloss', 'colsample_bylevel': 0.034570851765460854, 'depth': 12, 'random_strength': 1.3064973233129076, 'class_weights': [1, 1.25], 'n_estimators': 1000, 'l2_leaf_reg': 5.216846250136881, 'min_data_in_leaf': 1024, 'learning_rate': 0.03, 'max_bin': 127, 'used_ram_limit': '16gb', 'bootstrap_type': 'MVS', 'grow_policy': 'Depthwise'}
Fold 1 | Train Score = 0.91794 (0.9688, 0.8670)
Fold 1 | Val Score = 0.79359 (0.9220, 0.6652)
Clear cache 0
Fold 2 | Train Score = 0.91769 (0.9691, 0.8663)
Fold 2 | Val Score = 0.79064 (0.9233, 0.6580)
Clear cache 0



KeyboardInterrupt



In [121]:
df = study.trials_dataframe()

In [129]:
df["params_class_weights"] = df["params_class_weights"].astype(str)

In [130]:
df.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_bootstrap_type', 'params_class_weights',
       'params_colsample_bylevel', 'params_depth', 'params_l2_leaf_reg',
       'params_learning_rate', 'params_max_bin', 'params_min_data_in_leaf',
       'params_n_estimators', 'params_objective', 'params_random_strength',
       'params_subsample', 'state'],
      dtype='object')

In [145]:
df.groupby("params_subsample")["value"].mean()

params_subsample
0.600639    0.790795
0.769839    0.787349
0.795663    0.791467
Name: value, dtype: float64

In [113]:
study.best_trial

FrozenTrial(number=1, values=[0.7933261197848103], datetime_start=datetime.datetime(2022, 8, 19, 23, 9, 1, 229942), datetime_complete=datetime.datetime(2022, 8, 19, 23, 43, 0, 470587), params={'objective': 'Logloss', 'colsample_bylevel': 0.048626854325478676, 'depth': 11, 'bootstrap_type': 'MVS', 'random_strength': 1.2897439703972426, 'class_weights': [1, 1.5], 'n_estimators': 500, 'l2_leaf_reg': 4.176816737398486, 'min_data_in_leaf': 2048, 'learning_rate': 0.05, 'max_bin': 63}, distributions={'objective': CategoricalDistribution(choices=('Logloss',)), 'colsample_bylevel': LogUniformDistribution(high=0.1, low=0.03), 'depth': IntUniformDistribution(high=12, low=4, step=1), 'bootstrap_type': CategoricalDistribution(choices=('Bernoulli', 'MVS')), 'random_strength': UniformDistribution(high=2.0, low=1.0), 'class_weights': CategoricalDistribution(choices=([1, 1.25], [1, 1.5], [1, 2])), 'n_estimators': CategoricalDistribution(choices=(500, 750, 1000)), 'l2_leaf_reg': LogUniformDistribution(h

In [62]:
study_df = study.trials_dataframe()

In [11]:
# for col in get_cols(study_df, "params"):
#     sns.scatterplot(data=study_df, x=study_df[col], y=study_df["value"])
#     plt.show()

### Train Catboost

In [77]:
param = {
    "objective": "Logloss",
    "colsample_bylevel": 0.05,
    "depth": 6,
    "boosting_type": "Ordered",
    "bootstrap_type": "MVS",
    "n_estimators": 1000,
    "l2_leaf_reg": 3,
    "min_data_in_leaf": 1024,
    "learning_rate": 0.05,
    "max_bin": 127,
    "used_ram_limit": "16gb",
}

In [78]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1020)

In [79]:
train_score_list, val_score_list = [], []
for fold, (idx_tr, idx_va) in zip(range(1, 5+1), kf.split(train_agg, target)):
    fold = str(fold)
    X_train, y_train = train_agg.iloc[idx_tr], target[idx_tr]
    X_val, y_val = train_agg.iloc[idx_va], target[idx_va]
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model = cb.CatBoostClassifier(**param)
        model.fit(
            X_train, 
            y_train, 
            eval_set=[(X_val, y_val)], 
            verbose=100, 
            cat_features=cat_features,
            early_stopping_rounds=100
        )
    y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
    train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
    train_data, X_train, y_train = None, None, None
    y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
    val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
    valid_data, X_val, y_val = None, None, None
    train_score_list.append(train_score)
    val_score_list.append(val_score)
    print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
    print(f"Clear cache {gc.collect()}")

0:	learn: 0.6434646	test: 0.6435511	best: 0.6435511 (0)	total: 231ms	remaining: 3m 50s
100:	learn: 0.2661354	test: 0.2709502	best: 0.2709502 (100)	total: 15s	remaining: 2m 13s
200:	learn: 0.2598474	test: 0.2656707	best: 0.2656707 (200)	total: 29.7s	remaining: 1m 58s
300:	learn: 0.2562077	test: 0.2629458	best: 0.2629458 (300)	total: 44.2s	remaining: 1m 42s
400:	learn: 0.2535508	test: 0.2614296	best: 0.2614296 (400)	total: 59s	remaining: 1m 28s
500:	learn: 0.2514180	test: 0.2604717	best: 0.2604717 (500)	total: 1m 13s	remaining: 1m 13s
600:	learn: 0.2495896	test: 0.2598555	best: 0.2598555 (600)	total: 1m 28s	remaining: 58.7s
700:	learn: 0.2479837	test: 0.2593157	best: 0.2593157 (700)	total: 1m 43s	remaining: 44s
800:	learn: 0.2464774	test: 0.2588889	best: 0.2588889 (800)	total: 1m 58s	remaining: 29.3s
900:	learn: 0.2450839	test: 0.2586632	best: 0.2586565 (897)	total: 2m 13s	remaining: 14.6s
999:	learn: 0.2438038	test: 0.2584054	best: 0.2584054 (999)	total: 2m 27s	remaining: 0us

bestTest 

In [64]:
# experiment_dict = {}

In [80]:
# experiment_dict["fill_negative_127"] = {"train_scores": train_score_list, "val_scores": val_score_list}

In [49]:
np.mean(experiment_dict["no_impute"]["train_scores"]), np.mean(experiment_dict["no_impute"]["val_scores"])

(0.7618067142207205, 0.7390578755925745)

In [50]:
np.mean(experiment_dict["fill_zero"]["train_scores"]), np.mean(experiment_dict["fill_zero"]["val_scores"])

(0.7618417595776765, 0.7386932469726997)

In [66]:
np.mean(experiment_dict["fill_negative_one"]["train_scores"]), np.mean(experiment_dict["fill_negative_one"]["val_scores"])

(0.7618861119180279, 0.7385532351690534)

In [81]:
np.mean(experiment_dict["fill_negative_127"]["train_scores"]), np.mean(experiment_dict["fill_negative_127"]["val_scores"])

(0.7618463586373094, 0.7391923808237963)